# Exploration 7. 좋아할만한 다른 아티스트 찾기

<br/>

![img](https://lh3.googleusercontent.com/Ly5kt7brYp6KbLvw2-kBSeia_hNfmCsihTWUcSIPTwHZupkQKQrNQXUuaUZZoV-i6m-qI_P3QD2tzTt2jR2C_ErRTZjT6siw1-hYm8LqBzD7TerFFSLUmTlXpgvCQwv1MY3SmXRt)

<br/>

![img](https://lh5.googleusercontent.com/nP1xsjKM7L119rQcIjkfsURH5PVdYv-UmfqRsqEj3EjuV_eLXvuVyHK6N_k40zCzugcyrc_oT_DZzgppvHrodwMIy70M-rItWE985OI-GbY5iavR-MpLMzx3nYp6_Xt-613wUcaI)

lastfm은 영국의 인터넷 라디오 방송과 음악 추천 시스템(recommendation system)을 병합한 웹 사이트입니다. 이곳에서 제공하는 데이터 셋을 통해 한번 추천 알고리즘을 학습해봅시다.

먼저 필요한 라이브러리를 설치해줍시다!


In [1]:
!pip install implicit

     |████████████████████████████████| 1.1 MB 527 kB/s eta 0:00:01
  Created wheel for implicit: filename=implicit-0.4.2-cp37-cp37m-linux_x86_64.whl size=3102100 sha256=a483ead4964628a518f9c693b167fd29f7584bf434dd229ccab742fb2ba31c75
  Stored in directory: /home/aiffel0039/.cache/pip/wheels/2f/ed/a0/bcad2756c8a99e49d6edaf830ab604e5691910df4dd7cec870
Successfully built implicit


추천시스템은 이처럼 a)아이템은 매우 많고 b)유저의 취향은 다양할 때 유저가 소비할만한 아이템을 예측하는 모델입니다.

몇 가지 실제 사례를 보면,

- 유튜브 : 동영상이 매일 엄청나게 많이 올라오고 유저의 취향(게임 선호, 뷰티 선호, 지식 선호, 뉴스 선호)이 다양합니다.
- 페이스북 : 포스팅되는 글이 엄청 많고 유저가 관심 있는 페이지, 친구, 그룹은 전부 다릅니다.
- 아마존 : 카테고리를 한정해도 판매 품목이 엄청 많고 좋아하는 브랜드, 구매 기준이 다양합니다.

아티스트는 수없이 많고 새로운 아티스트가 계속 나오기 때문에, 유튜브 같은 추천 기반의 미디어 회사들은 꾸준히 아이템이 되는 아티스트 정보를 자동으로 수집하고, 사용자들이 아티스트를 선택한 정보를 업데이트하여 축적하고 있습니다. 이렇게 쌓인 방대한 데이터가 정확한 협업 필터링을 위한 기반이 됩니다.

오늘 우리는 이렇게 쌓인 데이터 사례 중 하나를 활용하여 유저가 좋아하는 특정 아티스트와 유사한 다른 아티스트를 추천하는 추천시스템을 만들어 보려고 합니다. Last.fm에서는 어떤 유저가 특정 아티스트의 노래를 몇 번이나 들었는지에 대한 데이터를 제공하고 있습니다. 2010년에 미국에서 서비스되고 있는 spotify에서 발생한 데이터이기 때문에 생소한 아티스트가 많을 수 있습니다.

그럼 작업 디렉토리를 생성하고 데이터셋을 다운로드 받아 실습 환경을 구성해봅시다. 

파일은 <http://millionsongdataset.com/lastfm/> 에서 다운로드 받을 수 있습니다. 다운로드 받은 파일은 `./data/lastfm-dataset-360K` 폴더에 저장합시다. 단, 파일 크기가 매우 커서 다운로드 하는데 시간이 꽤 걸립니다.

우리가 다룰 데이터는 `csv` 파일이 아닌 `tsv` 파일입니다. `tsv`는 Tab-Separated-Values의 약자로서, Comma-Separated Values의 약자인 `csv`와 거의 동일하나 구분자만 comma가 아닌 tab 문자를 사용한 것입니다.

`tsv` 파일도 마찬가지로 `pandas`를 활용해 열 수 있습니다. 

In [2]:
import pandas as pd
import os

fname = os.path.dirname(os.path.abspath('__file__'))
fname += r'/data/lastfm-dataset-360K/usersha1-artmbid-artname-plays.tsv'

col_names = ['user_id', 'artist_MBID', 'artist', 'play']

data = pd.read_csv(fname, sep='\t', names = col_names)
data.head(10)

,user_id,artist_MBID,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,3bd73256-3905-4f3a-97e2-8b341527f805,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,f2fb0ff0-5679-42ec-a55c-15109ce6e320,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,b3ae82c2-e60b-4551-a76d-6620f1b456aa,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,3d6bbeb7-f90e-4d10-b440-e153c0d10b53,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,bbd2ffd7-17f4-4506-8572-c1ea58c3f9a8,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,8bfac288-ccc5-448d-9573-c33ea2aa5c30,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,6531c8b1-76ea-4141-b270-eb1ac5b41375,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,21f3573f-10cf-44b3-aeaa-26cccd8448b5,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,c5db90c4-580d-4f33-b364-fbaa5a3a58b5,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,0639533a-0402-40ba-b6e0-18b067198b73,lunachicks,403


`artist_MBID` 항목은 우리의 관심사가 아닙니다. 사용하는 칼럼만 남기고 제거합시다.

In [3]:
# 사용하는 컬럼만 남겨줍니다.
using_cols = ['user_id', 'artist', 'play']
data = data[using_cols]
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die Ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔 줍시다.

In [4]:
data['artist'] = data['artist'].str.lower() # 검색을 쉽게하기 위해 아티스트 문자열을 소문자로 바꿔줍시다.
data.head(10)

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


첫 번째 유저가 어떤 아티스트의 노래를 듣는지 확인해 봅시다.

In [5]:
condition = data['user_id']== data.loc[0, 'user_id']
data.loc[condition]

,user_id,artist,play
0,00000c289a1829a808ac09c00daf10bc3c4e223b,betty blowtorch,2137
1,00000c289a1829a808ac09c00daf10bc3c4e223b,die ärzte,1099
2,00000c289a1829a808ac09c00daf10bc3c4e223b,melissa etheridge,897
3,00000c289a1829a808ac09c00daf10bc3c4e223b,elvenking,717
4,00000c289a1829a808ac09c00daf10bc3c4e223b,juliette & the licks,706
5,00000c289a1829a808ac09c00daf10bc3c4e223b,red hot chili peppers,691
6,00000c289a1829a808ac09c00daf10bc3c4e223b,magica,545
7,00000c289a1829a808ac09c00daf10bc3c4e223b,the black dahlia murder,507
8,00000c289a1829a808ac09c00daf10bc3c4e223b,the murmurs,424
9,00000c289a1829a808ac09c00daf10bc3c4e223b,lunachicks,403


처음 들어보는 아티스트가 많습니다. 추천시스템에서는 적용하는 분야에 대한 지식이 특히 더 필요한데, 모르면 검증을 못 하기 때문입니다.

## 데이터 탐색

추천 모델을 만들기 전에 데이터의 기본적인 정보를 보고 갑시다. 아래 항목들을 확인해 보고 싶습니다.

- 유저수, 아티스트수, 인기 많은 아티스트
- 유저들이 몇 명의 아티스트를 듣고 있는지에 대한 통계
- 유저 play 횟수 중앙값에 대한 통계
    
(참고) `pandas.DataFrame.nunique()`은 특정 컬럼에 포함된 유니크한 데이터의 개수를 알아보는데 유용합니다.



In [6]:
# 유저수
data['user_id'].nunique()

358868

In [7]:
# 아티스트 수
data['artist'].nunique()

291346

In [8]:
# 인기 많은 아티스트
artist_count = data.groupby('artist')['user_id'].count()
artist_count.sort_values(ascending=False).head(30)

artist
radiohead                77254
the beatles              76245
coldplay                 66658
red hot chili peppers    48924
muse                     46954
metallica                45233
pink floyd               44443
the killers              41229
linkin park              39773
nirvana                  39479
system of a down         37267
queen                    34174
u2                       33206
daft punk                33001
the cure                 32624
led zeppelin             32295
placebo                  32072
depeche mode             31916
david bowie              31862
bob dylan                31799
death cab for cutie      31482
arctic monkeys           30348
foo fighters             30144
air                      29795
the rolling stones       29754
nine inch nails          28946
sigur rós                28901
green day                28732
massive attack           28691
moby                     28232
Name: user_id, dtype: int64

In [9]:
# 유저별 몇명의 아티스트를 듣고 있는지에 대한 통계
user_count = data.groupby('user_id')['artist'].count()
user_count.describe()

count    358868.000000
mean         48.863234
std           8.524272
min           1.000000
25%          46.000000
50%          49.000000
75%          51.000000
max         166.000000
Name: artist, dtype: float64

In [10]:
# 유저별 play 횟수 중앙값에 대한 통계
user_median = data.groupby('user_id')['play'].median()
user_median.describe()

count    358868.000000
mean        142.187676
std         213.089902
min           1.000000
25%          32.000000
50%          83.000000
75%         180.000000
max       50142.000000
Name: play, dtype: float64

### 모델 검증을 위한 사용자 초기 정보 세팅

우리는 본인의 음악 취향과 가장 유사한 아티스트를 추천받고 싶습니다. 유튜브에선 이를 위해서 어떤 과정을 거쳤나요? 네, 사용자의 취향과 유사한 아티스트 정보를 5개 이상 입력받는 과정을 거쳤습니다.

우리도 동일한 과정이 필요합니다. 우리가 만든 추천시스템의 추후 검증 과정을 위해, '나는 A를 좋아한다.'라는 정보를 위 데이터셋에 5개 이상 추가해 주어야 합니다. 저는 아래와 같이 제가 좋아하는 유명한 아티스트들을 기존의 데이터에 추가해서 넣겠습니다.

In [11]:
# 본인이 좋아하는 아티스트 데이터로 바꿔서 추가하세요.
my_favorite = ['black eyed peas', 'maroon5', 'jason mraz', 'coldplay', 'beyoncé']

# 'zimin'이라는 user_id가 위 아티스트의 오래를 30회씩 들었다고 가정합시다.
my_playlist = pd.DataFrame({'user_id' : ['zimin']*5, 'artist':my_favorite, 'play' : [30]*5})

# user_id에 'zimin'이라는 데이터가 없다면 위에 임의로 만든 my_favortie 데이터를 추가해줍시다.
if not data.isin({'user_id' : ['zimin']})['user_id'].any():
    data = data.append(my_playlist)
    
data.tail(10)    

,user_id,artist,play
17535650,"sep 20, 2008",turbostaat,12
17535651,"sep 20, 2008",cuba missouri,11
17535652,"sep 20, 2008",little man tate,11
17535653,"sep 20, 2008",sigur rós,10
17535654,"sep 20, 2008",the smiths,10
0,zimin,black eyed peas,30
1,zimin,maroon5,30
2,zimin,jason mraz,30
3,zimin,coldplay,30
4,zimin,beyoncé,30


### 모델에 활용하기 위한 전처리 (실습)

사람이 태어나면 주민등록번호가, 학교에 가면 출석번호가 있듯이 데이터의 관리를 쉽게 하기 위해 번호를 붙여주고 싶습니다. 우리가 다루는 데이터에서는 user와 artist 각각에 번호를 붙이고 싶습니다. 보통 이런 작업을 **indexing**이라고 합니다. 추천시스템, 자연어처리에서 자주 하는 작업들이라 직접 해보시면 좋겠습니다.

(참고) `pandas.DataFrame.unique()`은 특정 컬럼에 포함된 유니크한 데이터만 모아 줍니다. indexing 작업을 위해 매우 유용합니다.

In [14]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = data['user_id'].unique()
artist_unique = data['artist'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v : k for k, v in enumerate(user_unique)}
artist_to_idx = {v : k for k, v in enumerate(artist_unique)}

In [15]:
# 인덱싱 제대로 되어있는지 확인
# 358869명의 유저 중 마지막으로 추가된 유저이니 358868이 나와야 합니다. 
print(user_to_idx['zimin'])
print(artist_to_idx['black eyed peas'])

358868
376


In [16]:
# indexing을 통해 데이터 칼럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16을 참고하세요

# user_to_idx.get을 통해 uesr_id 칼럼의 모든 값을 인덱싱한 Series를 구해봅시다.
# 혹시 정상적으로 인덱싱 되지 않은 row가 있다면 인덱스가 NaN이 될테니 dropna()로 제거합시다.
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()

if len(temp_user_data) == len(data): # 모든 row가 정상적으로 인덱싱 되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data # data['user_id']을 인덱싱된 Series로 교체해줍니다.
else:
    print("user_id column indexing fail!!")
    
# artist_to_idx을 통해 artist 칼럼도 동일한 방식으로 인덱싱 해줍니다.
temp_artist_data = data['artist'].map(artist_to_idx.get).dropna()
if len(temp_artist_data) == len(data):
    print('artist column indexing OK!!')
    data['artist'] = temp_artist_data
else:
    print('artist column indexing fail!!')
    
data.head(10)

user_id column indexing OK!!
artist column indexing OK!!


,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
5,0,5,691
6,0,6,545
7,0,7,507
8,0,8,424
9,0,9,403


In [17]:
data

,user_id,artist,play
0,0,0,2137
1,0,1,1099
2,0,2,897
3,0,3,717
4,0,4,706
...,...,...,...
0,358868,376,30
1,358868,270115,30
2,358868,3746,30
3,358868,62,30


`data`의 `user_id`와 `artist` 컬럼 내 값들이 모두 정수 인덱스 값으로 잘 변경되었나요? 이것으로 훈련을 위한 전처리가 완료되었습니다!!

## 사용자의 명시적/암묵적 평가

우리의 추천시스템은 사용자들이 아이템을 얼마나 선호하는지를 모델링하기를 원합니다. 그러자면 사용자의 아이템 선호도를 말해 주는 유저 행동 데이터셋이 필요합니다. 만약 우리 데이터가 좋아요나 별점처럼 선호도를 명시적(explicit)으로 나타내는 것이라면 참 좋겠지만, 지금 우리가 가지고 있는 데이터는 사용자가 아티스트의 곡을 몇번 플레이했나 하는 것 뿐입니다. 이렇게 서비스를 사용하면서 자연스럽게 발생하는 암묵적(implicit)인 피드백도 사용자의 아이템에 대한 평가를 알 수 있는 단서가 될 수 있습니다.

명시적/암묵적 피드백 데이터셋에 대해 잘 정리된 글을 한번 읽어봅시다.

- [Explicit vs Implicit Feedback Datasets](https://orill.tistory.com/entry/Explicit-vs-Implicit-Feedback-Datasets?category=1066301)

어떤 유저가 아티스트의 곡을 한번만 들었다고 생각해봅시다. 과연 이 유저는 이 아티스트를 좋아하는 걸까요 싫어하는 걸까요? 명확한 정답은 존재하지 않습니다. 이런 애매한 함묵적 데이터야 말로 도메인 지식과 직관이 활용되어야 하는 영역입니다.

그렇다면 1회만 플레이한 데이터의 비율은 어떻게 될까요? 한번 확인해봅시다.

In [18]:
# 1회만 play한 데이터의 비율을 보는 코드
only_one = data[data['play'] < 2]
one, all_data = len(only_one), len(data)

print(f'{one}, {all_data}')
print(f'Ratio of only_one over all data is {one/all_data:.2%}')  # f-format에 대한 설명은 https://bit.ly/2DTLqYU

147740, 17535660
Ratio of only_one over all data is 0.84%



우리가 앞으로 만들어갈 모델에서는 암묵적 데이터의 해석을 위해 다음과 같은 규칙을 적용할 것입니다.

```
1. 한번이라도 들었으면 선호한다고 판단한다.
2. 많이 재생한 아티스트에 대해 가중치를 두어서 더 확실히 좋아한다고 판단한다.
```

Matrix Factorization(MF)
데이터가 준비되었으니 이제 모델을 만들 차례입니다!

아래 그림의 왼쪽에서 보는 것처럼, 우리는 m명의 사용자들이 n명의 아티스트에 대해 평가한 데이터를 포함한 (m,n) 사이즈의 <u>평가행렬(Rating Matrix)</u>를 만들 것입니다. 행렬 중 일부는 데이터가 채워져 있지만, 나머지 부분은 데이터가 비어 있습니다. 우리가 앞에서 살펴보았던 추천시스템의 **`협업 필터링(Collaborative Filtering)`** 이란 결국은 이런 평가행렬을 전제로 하는 것입니다. 만약 우리가 이 평가행렬의 비어 있는 부분을 포함한 완벽한 정보를 얻을 수 만 있다면 (모든 사용자의 모든 아이템에 대한 선호도를 다 파악한 셈이니) 그야말로 완벽한 추천이 가능해질 것입니다.

우리는 추천 시스템의 다양한 모델 중 **Matrix Factorization(MF, 행렬분해)** 모델을 사용하겠습니다. `MF` 모델은 2006년 Netflix에서 백만달러의 상금을 걸고 개최한 자사 추천시스템의 성능을 10% 이상 향상시키는 챌린지를 계기로 알려지게 되었습니다. 기본 아이디어는 아래 그림과 같습니다. (m,n) 사이즈의 행렬 R을 (m,k) 사이즈의 행렬 P와 (k,n) 사이즈의 행렬 Q로 분해한다면 R이란 그저 P와 Q의 행렬곱으로 표현 가능할 수 있다는 간단한 아이디어입니다. 대체로 k는 m이나 n보다 훨씬 작은 값이기 때문에 계산량 측면으로도 훨씬 유리해집니다. 하지만 아이디어의 단순함에도 불구하고 MF 모델은 성능이 준수하고 Scalability가 좋아서 많이 사용되는 모델입니다.

![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-3-4.max-800x600.png)

아래 그림을 다시한번 봅시다. 위에 소개한 MF 모델의 개요도와 크게 다를 것은 없습니다. 단지, MF 모델을 사용자에게 영화를 추천하는 모델에 대입해서 그려 보았다는 정도의 차이가 있습니다. 위 그림 방식으로 말하자면 m=4, n=5, k=2인 MF 모델이 되겠군요.

위에서 MF 모델은 큰 평가행렬 R을 두개의 Feature Matrix P와 Q로 분해한다고 하였습니다.
무엇의 Feature일까요? (m, k) 사이즈의 Feature Matrix P는 k 차원의 벡터를 사용자 수만큼 모아놓은 행렬입니다. 그렇다면 아래 그림에서 첫번째 벡터 
P
0
=(1, 0.1)은 바로 빨간 모자를 쓴 첫번째 **사용자의 특성(Feature) 벡터**가 됩니다.
같은 방식으로 Q 행렬의 첫번째 벡터 
Q
0
=(0.9, -0.2)는 해리포터 **영화의 특성 벡터**가 되겠지요. MF 모델은 이 두 벡터를 내적해서 얻어지는 0.88이 바로 
R
0
,
0
 으로 정의되는 사용자의 영화 선호도로 보는 모델입니다.
 
![img](https://aiffelstaticprd.blob.core.windows.net/media/images/E-3-5.max-800x600.png)

모델의 목표는 모든 유저와 아이템에 대해 k-dimension의 벡터를 잘 만드는 것입니다.
벡터를 잘 만드는 기준은 유저i의 벡터(
U
i
)와 아이템j의 벡터(
I
j
)를 내적했을 때 유저i가 아이템j에 대해 평가한 수치(
M
i
j
)와 비슷한지 입니다.
U
i
⋅
I
j
=
M
i
j
MF에도 다양한 변형이 있습니다. 이번에 사용할 모델은 [Collaborative Filtering for Implicit Feedback Datasets](http://yifanhu.net/PUB/cf.pdf) 논문에서 제안한 모델을 사용할 것입니다.

이 모델의 경우 이전 스텝에서 제가 'black eyed peas'를 play했다는 데이터를 추가했기 때문에 저의 벡터와 black eyed peas의 벡터를 곱했을 때 1에 가까워야 모델이 잘 학습하는 것을 목표로 합니다.

오해할 수 있는데, 제가 black eyed peas를 5번 들었어도 모델이 근사하고자 하는 것이 <u>아티스트를 선호하는지 안 하는지를 맞추는 것</u>이기 때문에 두 벡터를 곱했을 때 5에 가까워지는 대신 1에 가까워져야 합니다. 이를 통해서 제가 들어보지 못한 queen에 대해서도 <u>저의 벡터와 queen의 벡터를 곱하여 수치를 예상할 수 있습니다.</u>

만약 목표가 유저의 재생횟수를 맞춰야 하는 것(혹은 별점을 맞추는 것)이라면 그에 맞는 다른 모델을 사용해야 합니다.

구조가 거의 똑같고 모델의 목표가 재생횟수를 맞춰지는 쪽으로 학습(유저와 아이템의 벡터를 만들기)하면 됩니다! 궁금하다면 [참고자료](https://towardsdatascience.com/recommendation-system-matrix-factorization-d61978660b4b)를 확인해주세요.

##  CSR(Compressed Sparse Row) Matrix

유저 X 아이템 평가행렬을 다시 생각해 봅시다. 유저는 36만 명이고 아티스트는 29만 명입니다. 이를 행렬로 표현하고 행렬의 각 원소에 정수 한 개 (1byte)가 들어간다면 36만 * 29만 * 1byte 
≈
 **97GB**가 필요합니다.

참고로 여러분들이 사용하고 계실 컴퓨터의 메모리는 많아야 16GB일 테니 97GB나 되는 거대한 행렬을 메모리에 올려 놓고 작업한다는 것은 불가능할 것입니다. 이런 경우의 좋은 대안이 되는 것이 **CSR(Compressed Sparse Row) Matrix**입니다. 다음 스텝에서 수행할 모델 학습의 input으로 사용할 데이터 타입을 CSR Matrix로 할 것입니다.


CSR Matrix는 Sparse한 matrix에서 0이 아닌 유효한 데이터로 채워지는 데이터의 값과 좌표 정보로만으로 구성하여 메모리 사용량을 최소화하면서도 Sparse한 matrix와 동일한 행렬을 표현할 수 있도록 하는 데이터 구조입니다. CSR에 대한 자세한 설명은 다음 링크를 참고해 주세요. 혹시 설명이 부족하면 그 아래 StackOverflow의 설명이 보완적인 역할을 해줄 것입니다.

- [Scipy sparse matrix handling](https://lovit.github.io/nlp/machine%20learning/2018/04/09/sparse_mtarix_handling/#csr-matrix)
- [StackOverflow csr_matrix 설명](https://stackoverflow.com/questions/53254104/cant-understand-scipy-sparse-csr-matrix-example/62118005#62118005)

이제 우리의 data를 CSR Matrix에 맞게 바꿔보겠습니다.

DataFrame을 만드는 방식이 다양하듯 csr_matrix를 [만드는 방법](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)은 다양하게 있습니다.
현재 data와 같은 데이터 구조에 적합한 방식은 위 링크에서 4번째로 설명하고 있는 방법입니다.

```python
csr_matrix((data, (row_ind, col_ind)), [shape=(M, N)])  
where data, row_ind and col_ind satisfy the relationship a[row_ind[k], col_ind[k]] = data[k]., M,N은 matrix의 shape
```

In [19]:
# 실습 위에 설명보고 이해해서 만들어보기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_artist = data['artist'].nunique()

csr_data = csr_matrix((data.play, (data.user_id, data.artist)), shape= (num_user, num_artist))
csr_data

<358869x291347 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Row format>

## MF 모델 학습하기

이전 스텝에서 설명한 *Matrix Factorization* 모델을 implicit 패키지를 사용하여 학습해 봅시다.

- `implicit` 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지입니다.
- 이 패키지에 구현된 `als(AlternatingLeastSquares)` 모델을 사용하겠습니다. `Matrix Factorization`에서 쪼개진 두 Feature Matrix를 한꺼번에 훈련하는 것은 잘 수렴하지 않기 때문에, 한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 *AlternatingLeastSquares* 방식이 효과적인 것으로 알려져 있습니다.

In [20]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분. 내용과는 무관하다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

AlternatingLeastSquares 클래스의 `__init__` 파라미터를 살펴보겠습니다. 1. factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지 2. regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지 3. use_gpu : GPU를 사용할 것인지 4. iterations : epochs와 같은 의미입니다. 데이터를 몇 번 반복해서 학습할 것인지

1,4를 늘릴수록 학습데이터를 잘 학습하게 되지만 과적합의 우려가 있으니 좋은 값을 찾아야 합니다.


In [21]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [22]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<291347x358869 sparse matrix of type '<class 'numpy.longlong'>'
	with 17535578 stored elements in Compressed Sparse Column format>

In [23]:
# 모델 훈련
als_model.fit(csr_data_transpose)

모델 학습이 끝났습니다!

모델이 1)저의 벡터와 black eyed peas의 **벡터를 어떻게 만들고 있는지**와 2)두 벡터를 **곱하면 어떤 값**이 나오는지 살펴보겠습니다.

In [24]:
zimin, black_eyed_peas = user_to_idx['zimin'], artist_to_idx['black eyed peas']
zimin_vector, black_eyed_peas_vector = als_model.user_factors[zimin], als_model.item_factors[black_eyed_peas]

print('슝=3')

슝=3


In [25]:
zimin_vector

array([ 1.3645555 , -0.07404073, -0.2215295 , -0.78069186,  0.24150316,
       -0.71990025,  0.6191671 ,  1.0096585 ,  0.04178312, -0.5763615 ,
        0.1086991 ,  0.06957515, -0.65086037,  0.594571  , -0.5083505 ,
        0.316356  , -0.14157763, -0.8874085 , -0.18926333,  0.6044974 ,
       -0.56908345, -0.5331353 ,  0.15996344, -1.2718129 ,  0.22032636,
        0.7696314 ,  0.75098455,  0.4780599 ,  0.2771798 ,  0.01355027,
        0.9966894 , -0.13249724, -0.31622177,  1.2728306 , -0.7243683 ,
        0.47222716, -0.03661947,  0.36875165,  0.70078766, -0.1328387 ,
        0.85753846, -0.5733329 , -0.4717844 , -0.90307075,  0.00535444,
        0.4232447 , -1.1904948 ,  1.987735  ,  0.09289998,  0.8759767 ,
       -1.0017465 ,  0.26722485, -0.6370124 ,  0.31473044, -0.6774048 ,
       -0.12331213, -0.47969633,  0.88099045,  0.46760648, -1.3826082 ,
        1.0719298 ,  1.090924  , -0.60184115, -0.7940398 , -0.98749924,
        0.95881814,  0.48601672,  0.4698665 ,  0.14208223, -0.55

In [26]:
black_eyed_peas_vector

array([ 0.02666401,  0.01762385, -0.0103195 ,  0.00765797,  0.01247407,
       -0.0122013 ,  0.00616536,  0.02499734,  0.00899297,  0.00165774,
        0.00296536, -0.01129847, -0.00010269, -0.0009027 , -0.00335912,
        0.01768456,  0.01054126, -0.0129554 ,  0.01633731,  0.02294127,
        0.01594604,  0.00075767,  0.00181682, -0.01479641,  0.01864399,
        0.0102895 ,  0.01321326,  0.00792573,  0.01159413,  0.00712834,
        0.0134917 ,  0.01595083,  0.00593655,  0.01591431, -0.00280397,
        0.02099051,  0.00039629,  0.00186478,  0.01280617,  0.00500935,
        0.01209986,  0.00646292, -0.00474733, -0.01339367,  0.0023232 ,
       -0.00329214,  0.00484227,  0.03090042,  0.01714546,  0.01544319,
       -0.00769379,  0.00539419,  0.00438083,  0.00797108, -0.00294586,
       -0.00050933, -0.00547677,  0.02870494,  0.00020766, -0.00071567,
        0.01391843,  0.01564428, -0.01164076,  0.00586949, -0.0035554 ,
        0.0001627 ,  0.0127365 ,  0.01532011,  0.00637251, -0.01

In [27]:
# zimin과 black_eyed_peas를 내적하는 코드
np.dot(zimin_vector, black_eyed_peas_vector)

0.524292

이런, 1이 나와야 될 것 같은데 한참 낮은 수치인 0.49정도가 나왔습니다. factors를 늘리거나 iterations를 늘려야할 것 같습니다. 이후 진행되는 내용은 일단 이렇게 학습된 모델을 사용하겠습니다.

그렇다면 모델이 저의 queen에 대한 선호도를 어떻게 예측할지 한 번 보겠습니다

In [28]:
queen = artist_to_idx['queen']
queen_vector = als_model.item_factors[queen]
np.dot(zimin_vector, queen_vector)

0.31086203

0.3 정도가 나왔습니다. queen에 대한 선호도 예측한 결과를 어떻게 해석하면 좋을까요?

그건 모델을 만드는 사람이 정하기 나름입니다. 객관적인 지표로 만들어서 기준을 정할수도 있고 도메인 경험을 통해 정할수도 있습니다.

추천시스템은 다른 머신러닝 task보다 객관적인 평가가 어려운 분야입니다. 사람을 도통 알 수 없기 때문에 객관적인 지표를 정하기 어렵습니다.


## 비슷한 아티스트 찾고 유저에게 추천하기

### 비슷한 아티스트 찾기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `similar_items` 메서드를 통하여 비슷한 아티스트를 찾습니다. 처음으로는 제가 좋아하는 `coldplay`로 찾아보겠습니다.

In [29]:
favorite_artist = 'coldplay'
artist_id = artist_to_idx[favorite_artist]
similar_artist = als_model.similar_items(artist_id, N=15)
similar_artist

[(62, 0.11803038),
 (277, 0.11628198),
 (5, 0.115498364),
 (28, 0.11477717),
 (473, 0.11428204),
 (217, 0.11377167),
 (490, 0.113518074),
 (247, 0.11259484),
 (418, 0.11236197),
 (694, 0.111570366),
 (910, 0.111251704),
 (268, 0.11110027),
 (55, 0.11072878),
 (782, 0.11019262),
 (1018, 0.11004274)]

(아티스트의 id, 유사도) Tuple 로 반환하고 있습니다. 아티스트의 id를 다시 아티스트의 이름으로 매핑시켜 주겠습니다.

In [30]:
#artist_to_idx 를 뒤집어, index로부터 artist 이름을 얻는 dict를 생성합니다. 
idx_to_artist = {v:k for k,v in artist_to_idx.items()}
[idx_to_artist[i[0]] for i in similar_artist]

['coldplay',
 'muse',
 'red hot chili peppers',
 'the killers',
 'placebo',
 'radiohead',
 'oasis',
 'the beatles',
 'u2',
 'foo fighters',
 'nirvana',
 'pink floyd',
 'arctic monkeys',
 'the white stripes',
 'the smashing pumpkins']

몇 번 더 반복해서 확인하기 위해 위의 코드를 함수로 만들고 확인해보겠습니다.

In [31]:
def get_similar_artist(artist_name: str):
    artist_id = artist_to_idx[artist_name]
    similar_artist = als_model.similar_items(artist_id)
    similar_artist = [idx_to_artist[i[0]] for i in similar_artist]
    return similar_artist

print("슝=3")

슝=3


저는 힙합도 좋아하기 때문에 `2pac`으로도 확인해 보겠습니다.

In [32]:
get_similar_artist('2pac')

['2pac',
 'snoop dogg',
 'notorious b.i.g.',
 'nas',
 'the game',
 'jay-z',
 'dmx',
 'dr. dre',
 'ludacris',
 '50 cent']

와 스눕독! 닥터드레!! 힙합의 경우 마니아들이 많아서 더 잘 추천되는 것으로 생각됩니다.

In [33]:
get_similar_artist('lady gaga')

['lady gaga',
 'britney spears',
 'rihanna',
 'katy perry',
 'beyoncé',
 'the pussycat dolls',
 'christina aguilera',
 'justin timberlake',
 'kelly clarkson',
 'pink']

lady gaga의 경우 여자 아티스트들이 추천되고 있습니다.



### 유저에게 아티스트 추천하기

`AlternatingLeastSquares` 클래스에 구현되어 있는 `recommend` 메서드를 통하여 제가 좋아할 만한 아티스트를 추천 받습니다. `filter_already_liked_items` 는 유저가 이미 평가한 아이템은 제외하는 Argument입니다.

In [34]:
user = user_to_idx['zimin']
# recommend에서는 user*item CSR Matrix를 받습니다.
artist_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
artist_recommended

[(274, 0.45945215),
 (350, 0.4544207),
 (550, 0.4479653),
 (1800, 0.44162065),
 (369, 0.4400596),
 (564, 0.43719316),
 (355, 0.41672456),
 (2249, 0.4059144),
 (409, 0.3988502),
 (5555, 0.3978703),
 (627, 0.39690974),
 (391, 0.3882465),
 (901, 0.38803935),
 (358, 0.37385565),
 (724, 0.37381542),
 (354, 0.3681873),
 (621, 0.36481193),
 (24, 0.3647449),
 (434, 0.36433005),
 (35, 0.35745338)]

In [35]:
[idx_to_artist[i[0]] for i in artist_recommended]

['michael jackson',
 'rihanna',
 'britney spears',
 'lady gaga',
 'justin timberlake',
 'kanye west',
 'madonna',
 'katy perry',
 'amy winehouse',
 'timbaland',
 'maroon 5',
 'christina aguilera',
 'pink',
 'kelly clarkson',
 'lily allen',
 'nelly furtado',
 'alicia keys',
 'jack johnson',
 'eminem',
 'john mayer']

rihanna를 추천해주고 있네요. 모델은 왜 rihanna를 추천해줬을까요? `AlternatingLeastSquares` 클래스에 구현된 `explain` 메소드를 사용하면 제가 기록을 남긴 데이터 중 *이 추천에 기여한 정도를 확인*할 수 있습니다.

In [36]:
rihanna = artist_to_idx['rihanna']
explain = als_model.explain(user, csr_data, itemid=rihanna)

In [38]:
[(idx_to_artist[i[0]], i[1]) for i in explain[1]]

[('beyoncé', 0.2229810324554575),
 ('black eyed peas', 0.13126272303783343),
 ('jason mraz', 0.05729093586438936),
 ('coldplay', 0.046167844074180966),
 ('maroon5', -0.0005617032797640577)]

beyoncé와 black eyed peas가 가장 크게 기여 했네요. 둘 다 흑인 아티스트이고 beyoncé의 경우는 같은 성별이라는 점도 크게 작용하지 않았을까 싶습니다.

눈치채신 분도 있겠지만 결과가 조금 이상합니다. `filter_already_liked_items=True`로 했는데 앞에 maroon 5는 제가 이미 평가한 아이템이 나오고 있습니다.이것은 implicit 버전 0.4.2에서 생긴 버그입니다. 코드는 사람이 만들기 때문에 이렇게 실수할 가능성이 있습니다.

오픈 소스 생태계에서는 누구든 이런 문제점을 지적하고 Pull Request를 통하여 수정을 제안할 수 있습니다.
많이들 사용하는 Tensorflow나 numpy, pandas도 여러 작은 버그들이 있습니다. 이런 유명한 패키지의 오류를 고쳐서 Pull Request를 하여 Contribute 한다면 좋은 이력이 되기도 합니다.

모델이 추천한 20개 중 몇 개를 실제로 선호하는지 계산하여 모델의 객관적인 지표를 만들 수 있을 것 같습니다. 자세한 방법은 [이쪽](https://danthetech.netlify.app/DataScience/evaluation-metrics-for-recommendation-system)을 참고해주세요.


---

## 마무리

추천시스템에서 Baseline으로 많이 사용되는 MF를 통해 유저에게 아티스트를 추천해보았습니다. 이제 여러분은 추천시스템 관련 task에서 기본 모델을 만드실 수 있을 겁니다.

그러나 우리가 만든 모델은 몇 가지 아쉬운 점이 있습니다.

1. 유저, 아티스트에 대한 Meta 정보를 반영하기가 쉽지 않습니다. 연령대별로 음악 취향이 다를수도 있잖아요!
2. 유저가 언제 play 했는지가 반영하기 쉽지 않습니다. 10년 전에 즐겨듣던 아티스트와 지금 즐겨듣는 아티스트를 비교해봅시다.

이러한 이유와 딥러닝의 발전으로 MF 이외의 모델 구조도 많이 연구/ 사용되고 있습니다. 하지만 어떤 추천 모델도 핵심은 MF와 비슷합니다.<br/>
유저와 아이템에 대한 벡터를 잘 학습하여 취향에 맞게(유저에 맞게) 아이템을 보여주거나(Retrieval) 걸러내는(Filtering) 역할입니다.